In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datacube import Datacube
import matplotlib.pyplot as plt
import sys
sys.path.append("../scripts")
from datacube.utils.cog import write_cog
from dea_bandindices import calculate_indices
from dea_datahandling import load_ard
from dea_dask import create_local_dask_cluster
from mpl_toolkits.mplot3d import Axes3D #to plot in 3D
import matplotlib.pyplot as plt         #to plot arrays as images
from matplotlib import colors           #extra colour representations
from matplotlib import cm               #extra colour representations
import numpy as np                      #to manipulate arrays
import cv2                              #to perform more complex manipulations on arrays
import geopandas as gpd

from dea_plotting import map_shapefile
from dea_spatialtools import xr_rasterize
import xarray as xr

/env/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
create_local_dask_cluster()
dc = Datacube(app = "")

Client Scheduler: tcp://127.0.0.1:37323 Dashboard: /proxy/8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.95 GB


/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


In [3]:
linescan_datasets = dc.find_datasets(product='linescan')
linescan_datasets = sorted(linescan_datasets, key = lambda ds: (ds.center_time, ds.id))
linescan_datasets = pd.Series(linescan_datasets)
indices = pd.read_csv('trainindices.csv')

In [4]:
indices

,ID,DD,gdf,train
0,I1,0.0,ABERFELDY_WEST_200_P1_201901260955_MGA94_55,23
1,I2,1.0,ABERFELDY_WEST_214_P1_201901261750_MGA94_55,26
2,I3,2.0,CREAM_JIM_JORDAN_217_P1_201901262218_MGA94_55,27
3,I4,5.0,JORDAN_231_P1_201901271500_MGA94_55,29
4,I5,6.0,JORDAN_234_P1_201901271901_MGA94_55,31
...,...,...,...,...
56,I57,58.0,YARRA51_633_P1_201903061644_MGA94_55,74
57,I58,59.0,YARRA51_704_P1_201903091659_MGA94_55,92
58,I59,60.0,YARRA51_726_P1_201903100129_MGA94_55,97
59,I60,61.0,YARRA51_794_P1_201903151412_MGA94_55,124


In [5]:
vector_file = 'resources/fire_boundaries.shp'
gdf = gpd.read_file(vector_file)
def clean_name(name):
    if name is None:
        res = None
    else:
        if name.upper()[-4::] == ".JPG":
            res = name.upper()[:-4].replace(' ','_')
        else:
            res = name.upper().replace(' ','_')
    return res
gdf['SourceNameClean'] = gdf.apply(lambda row: clean_name(row.SourceName), axis=1)
gdf.dtUTC = gdf.apply(lambda row: datetime.strptime(row.dtUTC, '%Y-%m-%d %H:%M:%S'), axis=1)
gdf.dtLocal = gdf.apply(lambda row: datetime.strptime(row.dtLocal, '%Y-%m-%d %H:%M:%S'), axis=1)

In [7]:
open_array = np.array([1,3,5,7,9])
close_array = np.array([25,36, 49, 64,81]) 
comb = []
for o in open_array:
    for c in close_array:
        comb.append((int(o),int(c)))

In [ ]:
for index, gdf_name in zip(indices.train, indices.gdf):
    dc = Datacube(app = "")
    src = dc.load(product='linescan', id=linescan_datasets[index].id, output_crs='epsg:28355', resolution=(-10,10))
    ob = gdf.loc[gdf.SourceNameClean == gdf_name]
    src['tgt'] = xr_rasterize(gdf=ob, da=src)
    src = src.isel(time = 0)
    crs = src.crs
    x_min, x_max, y_min, y_max = int(src.x.min().values), int(src.x.max().values), int(src.y.min().values), int(src.y.max().values)
    lon_range = (x_min, x_max)
    lat_range = (y_min, y_max)
    resolution = (-10, 10)

    query = {
        'x': lon_range,
        'y': lat_range,
        'time': ('2017','2019'),
        'measurements': [
            'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_2',
            'nbart_swir_1'
        ],
        'crs' : crs,
        'output_crs': crs,
        'resolution': (-10, 10),
        'group_by': 'solar_day',
    }
    history = load_ard(dc=dc, products=['ga_ls8c_ard_3','ga_ls7e_ard_3'], 
                              dask_chunks={'time': 1}, min_gooddata=0.85, ls7_slc_off=True, **query) 
    history = calculate_indices(history, index=['NDVI','MNDWI'], collection='ga_ls_3').median('time') 
    src = src.merge(history)
    del history

    lowerBound=100
    upperBound=255

    #create the mask over the top of our original image
    linescan_mask = cv2.inRange(src.linescan.values,lowerBound,upperBound)

    for i, (o, c) in enumerate(comb):

        #establish dialation and contraction parameters
        kernelOpen=np.ones((o,o)) # try it yourself!
        kernelClose=np.ones((c,c))    

        #denoise the pixels
        maskOpen=cv2.morphologyEx(linescan_mask,cv2.MORPH_OPEN,kernelOpen)

        maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

        src[f'maskOpen_{i}'] = (['y','x'], maskOpen)
        src[f'maskClose_{i}'] = (['y','x'], maskClose)
        
        del maskOpen
        del maskClose
    df = src.to_dataframe()
    del src
    del dc
    df = df.reset_index().drop(['time', 'spatial_ref'], axis = 1)
    tgt = df.tgt
    df.drop('tgt', axis = 1, inplace = True)
    df['tgt'] = tgt
    del tgt
    df.to_csv(f'./Datasets/train_{index}.csv', index = False)
    del df

In [7]:
test = pd.read_csv('test.csv')
fname = test.label.unique()

In [8]:
fname

array(['JORDAN_235_P1_201901281204_MGA94_55',
       'JORDAN_294_P1_201902011150_MGA94_55',
       'WALHALLA_313_P1_201902020733_MGA94_55',
       'WALHALLA_353_P1_201902031625_MGA94_55',
       'MACALISTER91_648_P1_201903070444_MGA94_55'], dtype=object)

In [27]:
list(enumerate(fname[2:]))

[(0, 'WALHALLA_313_P1_201902020733_MGA94_55'),
 (1, 'WALHALLA_353_P1_201902031625_MGA94_55'),
 (2, 'MACALISTER91_648_P1_201903070444_MGA94_55')]

In [28]:
for j, f in enumerate(fname[2:]):
    dc = Datacube(app = "")
    src = dc.load(product='linescan', label=f, output_crs='epsg:28355', resolution=(-10,10))
    src = src.isel(time = 0)
    crs = src.crs
    x_min, x_max, y_min, y_max = int(src.x.min().values), int(src.x.max().values), int(src.y.min().values), int(src.y.max().values)
    lon_range = (x_min, x_max)
    lat_range = (y_min, y_max)
    resolution = (-10, 10)

    query = {
        'x': lon_range,
        'y': lat_range,
        'time': ('2017','2019'),
        'measurements': [
            'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_2',
            'nbart_swir_1'
        ],
        'crs' : crs,
        'output_crs': crs,
        'resolution': (-10, 10),
        'group_by': 'solar_day',
    }
    history = load_ard(dc=dc, products=['ga_ls8c_ard_3','ga_ls7e_ard_3'], 
                              dask_chunks={'time': 1}, min_gooddata=0.85, ls7_slc_off=True, **query) 
    history = calculate_indices(history, index=['NDVI','MNDWI'], collection='ga_ls_3').median('time') 
    src = src.merge(history)
    del history
    lowerBound=80
    upperBound=255

    #create the mask over the top of our original image
    linescan_mask = cv2.inRange(src.linescan.values,lowerBound,upperBound)

    for i, (o, c) in enumerate(comb):

        #establish dialation and contraction parameters
        kernelOpen=np.ones((o,o)) # try it yourself!
        kernelClose=np.ones((c,c))    

        #denoise the pixels
        maskOpen=cv2.morphologyEx(linescan_mask,cv2.MORPH_OPEN,kernelOpen)

        maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

        src[f'maskOpen_{i}'] = (['y','x'], maskOpen)
        src[f'maskClose_{i}'] = (['y','x'], maskClose)

        del maskOpen
        del maskClose
    df = src.to_dataframe()
    del src
    df.reset_index(inplace = True)
    df.drop(['time','spatial_ref'], axis = 1, inplace = True)
    df[['x','y']] = df[['x','y']].astype(int)
    df.to_csv(f'test_{j+2}.csv', index = False)
    del df

/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,
../scripts/dea_datahandling.py:238: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn("Setting 'min_gooddata' percentage to > 0.0 "


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3
Counting good quality pixels for each time step
Filtering to 19 out of 116 time steps with at least 85.0% good quality pixels
Applying pixel quality/cloud mask
Returning 19 time steps as a dask array


/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,
../scripts/dea_datahandling.py:238: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn("Setting 'min_gooddata' percentage to > 0.0 "


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3
Counting good quality pixels for each time step
Filtering to 19 out of 116 time steps with at least 85.0% good quality pixels
Applying pixel quality/cloud mask
Returning 19 time steps as a dask array


/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,
../scripts/dea_datahandling.py:238: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn("Setting 'min_gooddata' percentage to > 0.0 "


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3
Counting good quality pixels for each time step
Filtering to 19 out of 115 time steps with at least 85.0% good quality pixels
Applying pixel quality/cloud mask
Returning 19 time steps as a dask array


In [ ]:
for f in fname:
    dc = Datacube(app = "")
    src = dc.load(product='linescan', label=f, output_crs='epsg:28355', resolution=(-10,10))
    src = src.isel(time = 0)
    crs = src.crs
    x_min, x_max, y_min, y_max = int(src.x.min().values), int(src.x.max().values), int(src.y.min().values), int(src.y.max().values)
    lon_range = (x_min, x_max)
    lat_range = (y_min, y_max)
    resolution = (-10, 10)
    
    query = {
        'x': lon_range,
        'y': lat_range,
        'time': ('2017','2019'),
        'measurements': [
            'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_2',
            'nbart_swir_1'
        ],
        'crs' : crs,
        'output_crs': crs,
        'resolution': (-10, 10),
        'group_by': 'solar_day',
    }
    history = load_ard(dc=dc, products=['ga_ls8c_ard_3','ga_ls7e_ard_3'], 
                              dask_chunks={'time': 1}, min_gooddata=0.85, ls7_slc_off=True, **query) 
    history = calculate_indices(history, index=['NDVI','MNDWI'], collection='ga_ls_3').median('time') 
    src = src.merge(history)
    del history
    lowerBound=80
    upperBound=255

    #create the mask over the top of our original image
    linescan_mask = cv2.inRange(src.linescan.values,lowerBound,upperBound)

    for i, (o, c) in enumerate(comb):

        #establish dialation and contraction parameters
        kernelOpen=np.ones((o,o)) # try it yourself!
        kernelClose=np.ones((c,c))    

        #denoise the pixels
        maskOpen=cv2.morphologyEx(linescan_mask,cv2.MORPH_OPEN,kernelOpen)

        maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

        src[f'maskOpen_{i}'] = (['y','x'], maskOpen)
        src[f'maskClose_{i}'] = (['y','x'], maskClose)

        del maskOpen
        del maskClose
    df = src.to_dataframe()
    del src
    df.reset_index(inplace = True)
    df.drop(['time','spatial_ref'], axis = 1, inplace = True)
    df[['x','y']] = df[['x','y']].astype(int)
    


In [11]:
n = 0

In [12]:
linescan_names = [linescan_datasets[x].metadata_doc['label'] for x in range(len(linescan_datasets))]
linescan_names = pd.Series(linescan_names)

In [13]:
src = dc.load(product='linescan', label=fname[n], output_crs='epsg:28355', resolution=(-10,10))
src

<xarray.Dataset>
Dimensions:      (time: 1, x: 2870, y: 2507)
Coordinates:
  * time         (time) datetime64[ns] 2019-01-28T01:04:00
  * y            (y) float64 5.841e+06 5.841e+06 ... 5.816e+06 5.816e+06
  * x            (x) float64 4.211e+05 4.212e+05 ... 4.498e+05 4.498e+05
    spatial_ref  int32 28355
Data variables:
    linescan     (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
Attributes:
    crs:           epsg:28355
    grid_mapping:  spatial_ref

In [75]:
dc = Datacube(app = "")
src = dc.load(product='linescan', label=fname[n], output_crs='epsg:28355', resolution=(-10,10))
src = src.isel(time = 0)
crs = src.crs
x_min, x_max, y_min, y_max = int(src.x.min().values), int(src.x.max().values), int(src.y.min().values), int(src.y.max().values)
lon_range = (x_min, x_max)
lat_range = (y_min, y_max)
resolution = (-10, 10)


/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


In [76]:
query = {
    'x': lon_range,
    'y': lat_range,
    'time': ('2017','2019'),
    'measurements': [
        'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_2',
        'nbart_swir_1'
    ],
    'crs' : crs,
    'output_crs': crs,
    'resolution': (-10, 10),
    'group_by': 'solar_day',
}
history = load_ard(dc=dc, products=['ga_ls8c_ard_3','ga_ls7e_ard_3'], 
                          dask_chunks={'time': 1}, min_gooddata=0.85, ls7_slc_off=True, **query) 
history = calculate_indices(history, index=['NDVI','MNDWI'], collection='ga_ls_3').median('time') 
src = src.merge(history)
del history
lowerBound=80
upperBound=255

#create the mask over the top of our original image
linescan_mask = cv2.inRange(src.linescan.values,lowerBound,upperBound)

for i, (o, c) in enumerate(comb):

    #establish dialation and contraction parameters
    kernelOpen=np.ones((o,o)) # try it yourself!
    kernelClose=np.ones((c,c))    

    #denoise the pixels
    maskOpen=cv2.morphologyEx(linescan_mask,cv2.MORPH_OPEN,kernelOpen)

    maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

    src[f'maskOpen_{i}'] = (['y','x'], maskOpen)
    src[f'maskClose_{i}'] = (['y','x'], maskClose)

    del maskOpen
    del maskClose


../scripts/dea_datahandling.py:238: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn("Setting 'min_gooddata' percentage to > 0.0 "


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3
Counting good quality pixels for each time step
Filtering to 19 out of 115 time steps with at least 85.0% good quality pixels
Applying pixel quality/cloud mask
Returning 19 time steps as a dask array


In [77]:
df = src.to_dataframe()
del src
df.reset_index(inplace = True)
df.drop(['time','spatial_ref'], axis = 1, inplace = True)
df[['x','y']] = df[['x','y']].astype(int)
df.head()

,x,y,linescan,nbart_blue,nbart_green,nbart_red,nbart_nir,nbart_swir_2,nbart_swir_1,NDVI,...,maskOpen_11,maskClose_11,maskOpen_12,maskClose_12,maskOpen_13,maskClose_13,maskOpen_14,maskClose_14,maskOpen_15,maskClose_15
0,458585,5848585,0,182.0,230.0,227.0,1395.0,308.0,661.0,0.714469,...,0,0,0,0,0,0,0,0,0,0
1,458585,5848575,0,182.0,230.0,227.0,1395.0,308.0,661.0,0.714469,...,0,0,0,0,0,0,0,0,0,0
2,458585,5848565,0,182.0,230.0,227.0,1395.0,308.0,661.0,0.714469,...,0,0,0,0,0,0,0,0,0,0
3,458585,5848555,0,168.0,228.0,209.0,1616.0,341.0,726.0,0.743258,...,0,0,0,0,0,0,0,0,0,0
4,458585,5848545,0,168.0,228.0,209.0,1616.0,341.0,726.0,0.743258,...,0,0,0,0,0,0,0,0,0,0


In [78]:
df = df.loc[df.linescan.gt(0),:]

In [16]:
df

NameError: name 'df' is not defined